# Data Analysis

### Standard imports and loading data

In [151]:
import pandas as pd
import numpy as np
from PayneLabData import BurkholderiaTimeCourse as btc
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.palettes import Viridis8
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

## B_thai No Glucose Analysis

In [35]:
bthai = btc.bthai_raw

### Find standard biological variability by using the standard deviations of each time point

In [83]:
bthai_stdev = pd.DataFrame()
bthai_stdev['tp1'] = np.log(bthai[['Glc(-)_T11', 'Glc(-)_T12', 'Glc(-)_T13', 'Glc(-)_T14']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp2'] = np.log(bthai[['Glc(-)_T21', 'Glc(-)_T22', 'Glc(-)_T23', 'Glc(-)_T24']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp3'] = np.log(bthai[['Glc(-)_T31', 'Glc(-)_T32', 'Glc(-)_T33', 'Glc(-)_T34']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp4'] = np.log(bthai[['Glc(-)_T41', 'Glc(-)_T42', 'Glc(-)_T43', 'Glc(-)_T44']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp5'] = np.log(bthai[['Glc(-)_T51', 'Glc(-)_T52', 'Glc(-)_T53', 'Glc(-)_T54']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev = bthai_stdev.dropna(axis = 0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


### Find the typical biological variability--the number below which 99% of standard deviations (across all time points) lie

In [150]:
all_stdev = pd.melt(bthai_stdev)['value'].tolist()
all_stdev.sort()
logcutoff = all_stdev[int(len(all_stdev)*0.99)]
cutoff = 2**logcutoff
print(cutoff)

1.4908900090837327


In [158]:
# Build the basic figure
std_p = figure(width=1000, plot_height=1000, title = 'Distribution of Standard Deviations at Each Time Point',
                  x_axis_label = 'Standard Deviation (Log Scale)',
                  y_axis_label = 'Number of Proteins')

# Create the histograms for each time point
tp1_std_hist, edges1 = np.histogram(bthai_stdev['tp1'], density=False, bins=100)
tp2_std_hist, edges2 = np.histogram(bthai_stdev['tp2'], density=False, bins=100)
tp3_std_hist, edges3 = np.histogram(bthai_stdev['tp3'], density=False, bins=100)
tp4_std_hist, edges4 = np.histogram(bthai_stdev['tp4'], density=False, bins=100)
tp5_std_hist, edges5 = np.histogram(bthai_stdev['tp5'], density=False, bins=100)

# Add each histogram to the graph
for data, edge, name, color in zip([tp1_std_hist, tp2_std_hist, tp3_std_hist, tp4_std_hist, tp5_std_hist], [edges1, edges2, edges3, edges4, edges5], ['Time Point 1', 'Time Point 2', 'Time Point 3', 'Time Point 4', 'Time Point 5'], Viridis8):
    std_p.quad(top=data, bottom=0, left=edge[:-1], right=edge[1:], color=color, fill_alpha = 0.25, legend=name)

# Add significance cutoff line
vline = Span(location = logcutoff, dimension = 'height', line_width=2)
std_p.renderers.extend([vline])

# Plot Styling
std_p.xgrid.visible = False
std_p.ygrid.visible = False
std_p.xaxis.minor_tick_line_color = None
std_p.yaxis.minor_tick_line_color = None
std_p.legend.click_policy = 'hide'
std_p.title.text_font_size = '24pt'
std_p.title.align = 'center'
std_p.xaxis.axis_label_text_font_size = '20pt'
std_p.yaxis.axis_label_text_font_size = '20pt'

show(std_p)

## Calculate the means for each time point

In [36]:
bthai_mean = pd.DataFrame()
bthai_mean['tp1'] = bthai[['Glc(-)_T11', 'Glc(-)_T12', 'Glc(-)_T13', 'Glc(-)_T14']].mean(axis=1)
bthai_mean['tp2'] = bthai[['Glc(-)_T21', 'Glc(-)_T22', 'Glc(-)_T23', 'Glc(-)_T24']].mean(axis=1)
bthai_mean['tp3'] = bthai[['Glc(-)_T31', 'Glc(-)_T32', 'Glc(-)_T33', 'Glc(-)_T34']].mean(axis=1)
bthai_mean['tp4'] = bthai[['Glc(-)_T41', 'Glc(-)_T42', 'Glc(-)_T43', 'Glc(-)_T44']].mean(axis=1)
bthai_mean['tp5'] = bthai[['Glc(-)_T51', 'Glc(-)_T52', 'Glc(-)_T53', 'Glc(-)_T54']].mean(axis=1)
bthai_mean = bthai_mean.dropna(axis = 0)

## Cluster the proteins (manually) based on increasing/decreasing/no change/spike at time point

In [154]:
cutoff_95 = 0.9007950046165919
cutoff_99 = 1.7792177941476046

bthai_mean['max'] = bthai_mean[['tp1', 'tp2', 'tp3', 'tp4', 'tp5']].max(axis=1)
bthai_mean['min'] = bthai_mean[['tp1', 'tp2', 'tp3', 'tp4', 'tp5']].min(axis=1)
bthai_mean['diff_max_min'] = bthai_mean['max'] - bthai_mean['min']
bin_rejects = bthai_mean.loc[bthai_mean['diff_max_min'] < cutoff]
significant_change = bthai_mean.loc[bthai_mean['diff_max_min'] >= cutoff]
significant_change['max'] = significant_change[['tp1', 'tp2', 'tp3', 'tp4', 'tp5']].max(axis=1)

bin_1 = significant_change[significant_change['tp1'] == significant_change['max']]
bin_2 = significant_change[significant_change['tp2'] == significant_change['max']]
bin_3 = significant_change[significant_change['tp3'] == significant_change['max']]
bin_4 = significant_change[significant_change['tp4'] == significant_change['max']]
bin_10 = significant_change[significant_change['tp5'] == significant_change['max']]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Create a simple chart to show the number of proteins in each category

In [187]:
bins = ['Peak TP1', 'Peak TP2', 'Peak TP3', 'Peak TP4', 'Peak TP5']
counts = [len(bin_1), len(bin_2), len(bin_3), len(bin_4), len(bin_10)]

p = figure(x_range = bins, plot_height = 500, title = 'Distribution of Protein Peaks',
           x_axis_label = 'Time Point Peak',
           y_axis_label = 'Number of Proteins')

p.vbar(x = bins, top = counts, width = 0.9, color = Viridis8[0:5])

p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '20pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '15pt'
p.yaxis.axis_label_text_font_size = '15pt'
p.yaxis.minor_tick_line_color = None

show(p)

## Plot each category

In [155]:
p = figure(plot_width=400, plot_height=400, title='Proteins With No Significant Change Over Time')

for i in range(0, len(bin_rejects)):
    p.line([1, 2, 3, 4, 5], [bin_rejects['tp1'][i], bin_rejects['tp2'][i], bin_rejects['tp3'][i], bin_rejects['tp4'][i], bin_rejects['tp5'][i]], color='red')
    
p.xaxis.axis_label = 'Time Point'
p.yaxis.axis_label = 'Relative Protein Abundance'
p.xgrid.visible = False
p.ygrid.visible = False
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [181]:
p = figure(plot_width=1000, plot_height=1000, title = 'Maximum Abundance at Time Point 1 (n = ' + str(len(bin_1)) + ')',
           x_axis_label = 'Time Point',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_1)):
    p.line([1, 2, 3, 4, 5], [bin_1['tp1'][i], bin_1['tp2'][i], bin_1['tp3'][i], bin_1['tp4'][i], bin_1['tp5'][i]], color='teal')
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '24pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [182]:
p = figure(plot_width=1000, plot_height=1000, title = 'Maximum Abundance at Time Point 2 (n = ' + str(len(bin_2)) + ')',
           x_axis_label = 'Time Point',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_2)):
    p.line([1, 2, 3, 4, 5], [bin_2['tp1'][i], bin_2['tp2'][i], bin_2['tp3'][i], bin_2['tp4'][i], bin_2['tp5'][i]], color='purple')
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '24pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [183]:
p = figure(plot_width=1000, plot_height=1000, title = 'Maximum Abundance at Time Point 3 (n = ' + str(len(bin_3)) + ')',
           x_axis_label = 'Time Point',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_3)):
    p.line([1, 2, 3, 4, 5], [bin_3['tp1'][i], bin_3['tp2'][i], bin_3['tp3'][i], bin_3['tp4'][i], bin_3['tp5'][i]], color='mediumseagreen')
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '24pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [184]:
p = figure(plot_width=1000, plot_height=1000, title = 'Maximum Abundance at Time Point 4 (n = ' + str(len(bin_4)) + ')',
           x_axis_label = 'Time Point',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_4)):
    p.line([1, 2, 3, 4, 5], [bin_4['tp1'][i], bin_4['tp2'][i], bin_4['tp3'][i], bin_4['tp4'][i], bin_4['tp5'][i]], color='mediumvioletred')
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '24pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [185]:
p = figure(plot_width=1000, plot_height=1000, title = 'Maximum Abundance at Time Point 5 (n = ' + str(len(bin_10)) + ')',
           x_axis_label = 'Time Point',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_10)):
    p.line([1, 2, 3, 4, 5], [bin_10['tp1'][i], bin_10['tp2'][i], bin_10['tp3'][i], bin_10['tp4'][i], bin_10['tp5'][i]])
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '24pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)